In [ ]:
# Minimal bootstrap with Capella Management API
import os, time, requests

BASE_URL = "https://cloudapi.cloud.couchbase.com"
TOKEN = "QkhFdFZzZVc3ZUlXczQ3M292a0trMGt0NnpTTTI3U246Snd0UU9FcGZRdn5ZNXJpaWJsd19Bc0BUWEx4K2NCT0RXQHFKVUA0flFUMCNXMkhlX0E5XzE4NENxUnFRc1M/SQ=="   # from Capella UI → API Key (Org Owner role)
ORG_ID = ""                         # leave empty to auto-pick first org
PROJECT_NAME = "agentic-lab"
CLUSTER_NAME = "agentic-lab-free"

s = requests.Session()
s.headers.update({"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"})
api = lambda m,p,**k: s.request(m, f"{BASE_URL}{p}", **k).json()

# 1. Organization
if not ORG_ID:
    ORG_ID = api("GET", "/v4/organizations")["data"][0]["id"]
print("Org:", ORG_ID)

# 2. Project (get or create)
projects = api("GET", f"/v4/organizations/{ORG_ID}/projects")["data"]
proj = next((p for p in projects if p["name"]==PROJECT_NAME), None)
if proj: project_id = proj["id"]
else: project_id = api("POST", f"/v4/organizations/{ORG_ID}/projects", json={"name": PROJECT_NAME})["id"]
print("Project:", project_id)

# 3. Cluster (free tier; reuse if exists)
clusters = api("GET", f"/v4/organizations/{ORG_ID}/projects/{project_id}/clusters")["data"]
cl = next((c for c in clusters if c["name"]==CLUSTER_NAME), None)
if cl: cluster_id = cl["id"]
else:
    body = {"name": CLUSTER_NAME,
            "cloudProvider":{"type":"aws","region":"us-east-2","cidr":"10.1.30.0/23"}}
    response = api("POST", f"/v4/organizations/{ORG_ID}/projects/{project_id}/clusters/freeTier", json=body)
    print("API Response:", response) # Add this line to print the response
    cluster_id = response["id"]
print("Cluster:", cluster_id)

# Wait until cluster ready
for _ in range(40):
    status = api("GET", f"/v4/organizations/{ORG_ID}/projects/{project_id}/clusters/{cluster_id}").get("currentState","")
    print("Cluster status:", status)
    if status.lower() in ["healthy","ready","running"]: break
    time.sleep(30)

# 4. Import travel-sample
buckets = api("GET", f"/v4/organizations/{ORG_ID}/projects/{project_id}/clusters/{cluster_id}/buckets")["data"]
if not any(b["name"]=="travel-sample" for b in buckets):
    try:
        api("POST", f"/v4/organizations/{ORG_ID}/projects/{project_id}/clusters/{cluster_id}/sampleBuckets",
            json={"name":"travel-sample"})
        print("Requested travel-sample import.")
    except Exception as e:
        print("Sample import not supported in this tenant. Use UI to add travel-sample.")
else:
    print("travel-sample already present.")

Org: 2b507c00-bf9a-4839-bbf2-a36618414089
Project: 51806e83-2a20-422b-9c30-7fa1347015d3
Cluster: b31d7641-24d1-4393-bd01-dccfdd4f4d9f
Cluster status: healthy
travel-sample already present.


In [ ]:
# Get cluster information and print the full response
cluster_info = api("GET", f"/v4/organizations/{ORG_ID}/projects/{project_id}/clusters/{cluster_id}")
print("Full Cluster API Response:")
print(cluster_info)

# You can also specifically check the 'status' key from the response
# if isinstance(cluster_info, dict):
#     status = cluster_info.get("status", "Status key not found in response")
#     print("\nCluster status from API:", status)
# else:
#     print("\nAPI response is not a dictionary.")

Full Cluster API Response:
{'audit': {'createdAt': '2025-09-16T21:02:13.579595203Z', 'createdBy': 'BHEtVseW7eIWs473ovkKk0kt6zSM27Sn', 'modifiedAt': '2025-09-16T21:05:05.165824861Z', 'modifiedBy': 'CP-INTERNAL-API-SYSTEM-USER', 'version': 9}, 'availability': {'type': 'single'}, 'cloudProvider': {'cidr': '10.1.30.0/23', 'region': 'us-east-2', 'type': 'aws'}, 'configurationType': 'singleNode', 'connectionString': 'cb.x92tww-9pj2w-iry.cloud.couchbase.com', 'couchbaseServer': {'version': '7.6.7'}, 'currentState': 'healthy', 'description': '', 'enablePrivateDNSResolution': False, 'id': 'b31d7641-24d1-4393-bd01-dccfdd4f4d9f', 'name': 'agentic-lab-free', 'serviceGroups': [{'node': {'compute': {'cpu': 2, 'ram': 8}, 'disk': {'iops': 3000, 'storage': 10, 'type': 'gp3'}}, 'numOfNodes': 1, 'services': ['search', 'index', 'data', 'query']}], 'support': {'plan': '', 'timezone': 'PT'}}
